(ch_remote)=
# 在远程计算机上运行

将在各种硬件平台上运行和优化程序。一种方法是使用所需的硬件登录到计算机，安装所需的包，然后在那里运行工作负载。然而，这使得维护源代码和数据变得困难，特别是当目标硬件的功耗最小时。在本节中，我们将描述另一种解决方案：在远程机器上运行守护进程，然后将编译后的模块和输入数据发送给它以供执行。

In [1]:
from tvm_book.contrib import d2ltvm
import numpy as np
import tvm
from tvm import te, rpc, relay
from PIL import Image

注意，从 TVM 导入 `rpc` 模块。[RPC](https://en.wikipedia.org/wiki/Remote_procedure_call)，即远程过程调用（remote procedure call），允许在远程位置上执行程序。

## 设置远程计算机

需要在远程机器上安装 TVM `runtime` 模块。安装设置几乎与 TVM 相同（请参阅 {ref}`ch_install`），但只需要构建运行时，即 `make runtime`，而不是整个 TVM 库。运行时大小通常小于 1MB，这使得它适用于具有内存限制的设备。如有必要，还需要启用适当的后端，例如 `CUDA` 或 `OpenCL`。

一旦安装了运行时，就可以通过在远程机器上运行以下命令来启动 daemon。

```bash
python -m tvm.exec.rpc_server --host 0.0.0.0 --port=9090
```

它将启动 RPC 服务器，该服务器绑定本地 9090 端口进行侦听。应该看到以下输出，表明服务器已经启动。

```bash
INFO:RPCServer:bind to 0.0.0.0:9090
```

此外，需要在远程机器上检查两件事情。

- 一个是远程机器的 IP。在 Linux 或 macOS 上，你可以通过 `ifconfig  | grep inet` 获取它。如果有防火墙，也要记得打开 9090 端口。
- 另一个是目标架构。这对于 GPU 来说很简单，我们稍后会讨论它。对于 CPU，最简单的方法是在远程机器上安装 LLVM，然后检查 `llvm-config --host-target`。我们所使用的远程机器的返回值是 `x86_64-pc-linux-gnu`。

此 target 三元组具有一般格式 `<arch><sub>-<vendor>-<sys>-<abi>`，这里

- arch: x86, x86_64, arm, thumb, mips, etc.
- sub: for ARM, there are v5, v6m, v7a, v7m, v8, etc.
- vendor: pc, apple, nvidia, ibm, etc.
- sys: linux, win32, darwin, cuda, none, unknown, etc.
- abi: eabi, gnu, android, macho, elf, etc.

例如，`x86_64-apple-darwin17.7.0` 即 MacbookPro，而 `armv6k-unknown-linux-gnueabihf` 指的是 Raspberry Pi 4B.


## 编译远程计算机的程序

在远程机器上运行 {ref}`ch_vector_add_te`。注意，通过 LLVM 的 `-target` 参数指定了远程机器目标。

In [2]:
local_demo = True

n = 100
args = d2ltvm.vector_add(n)

if local_demo:
    target = "llvm"
else:
    target = "llvm -target=x86_64-linux-gnu"
s = te.create_schedule(args[-1].op)
rt_mod = tvm.build(s, args, target)

```{note}
在这里使用 `'llvm'` 来使这个教程可以在网页构建服务器上运行。要在真正的远程设备上运行本教程，请将 `local_demo` 改为 `False`，并将 `target` 改为适合你设备的目标。
```

然后将编译后的模块保存到磁盘，稍后将其上传到远程计算机。

In [3]:
from tvm.contrib.utils import TempDirectory


class TempModule(TempDirectory):
    """Create temp dir which deletes the contents when exit.

    Parameters
    ----------
    custom_path : str, optional
        Manually specify the exact temp dir path

    keep_for_debug : bool
        Keep temp directory for debugging purposes
    Returns
    -------
    temp : TempDirectory
        The temp directory object
    """
    def export_library(self, module, filename, *args, **kwargs):
        """将该 module 保存在本地临时文件夹中
        """
        path = self.relpath(filename)
        module.export_library(path, *args, **kwargs)
        return path

    def upload(self, remote, module, filename):
        """将 module 上传至 remote 并返回远程模块"""
        path = self.export_library(module, filename)
        remote.upload(path)
        # remote_module
        return remote.load_module(filename)

In [4]:
temp_module = TempModule()
mod_fname = 'vector-add.tar'
mod_path = temp_module.export_library(rt_mod, mod_fname)

## 在远程计算机上评估

使用之前检查的 IP 连接到远程机器。

In [5]:
if local_demo:
    remote = rpc.LocalSession()
else:
    # 将其改为你的目标设备的 IP 地址
    host = "10.77.1.162"
    port = 9090
    remote = rpc.connect(host, port)

将编译好的库上传至远程机器，并将其加载到远程机器的内存中。

In [6]:
remote.upload(mod_path)
remote_mod = remote.load_module(mod_fname)

现在 `remote_mod` 是远程模块对象。

在创建数据时，将设备上下文指定为远程机器上的 CPU。与之前一样，数据将在本地机器上创建，但稍后将发送到远程机器。注意，使用 NumPy 创建数据，但是不需要让远程机器也安装 NumPy。

In [7]:
ctx = remote.cpu()
a, b, c = d2ltvm.get_abc(n, lambda x: tvm.nd.array(x, ctx))

由于数据和库在远程机器上都已经准备好了，让我们也在其上执行程序。

In [8]:
remote_mod(a, b, c)

最后，`.numpy` 方法将数据发送回本地机器，并转换为 NumPy 数组。所以可以像之前一样验证结果。

In [9]:
np.testing.assert_equal(a.asnumpy()+b.asnumpy(), c.asnumpy())

## 运行神经网络推理

在远程机器上运行 {mod}`ch_from_mxnet` 中使用的 ResNet-18。和前面一样，加载样本图像和 Imagenet 1K 标签。

In [10]:
image = Image.open('../data/cat.jpg').resize((224, 224))
x = d2ltvm.image_preprocessing(image)
with open('../data/imagenet1k_labels.txt') as f:
    labels = eval(f.read())

然后，变换、编译和保存模块。注意，只需要将包含编译的算子共享库保存到磁盘。

In [11]:
from mxnet.gluon.model_zoo.vision import get_model

mod_fname = 'resnet18.tar'
model = get_model("resnet18_v2", pretrained=True)
relay_mod, relay_params = relay.frontend.from_mxnet(model, {'data': x.shape})
with tvm.transform.PassContext(opt_level=3):
    rt_mod = relay.build(relay_mod, target, params=relay_params)
mod_path = temp_module.export_library(rt_mod, mod_fname)

/media/workspace/anaconda3/envs/mxnetx/lib/python3.10/site-packages/tvm/driver/build_module.py:263: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


接下来，将保存的库上传到远程机器，并将其加载到内存中。然后，可以使用模型定义、远程库和远程上下文创建运行时。

In [12]:
from tvm.contrib import graph_executor

remote.upload(mod_path)
remote_mod = remote.load_module(mod_fname)
remote_fuc = remote_mod["default"]
remote_rt =  graph_executor.GraphModule(remote_fuc(ctx))

其中参数和输入都在本地机器上。运行时将正确地将它们 upload 到远程计算机。

In [14]:
remote_rt.run(data=tvm.nd.array(x))
scores = remote_rt.get_output(0).asnumpy()[0]
scores.shape
a = np.argsort(scores)[-1:-5:-1]
labels[a[0]], labels[a[1]]

('tiger cat', 'Egyptian cat')

## 小结

- 可以在远程机器上安装 TVM 运行时来启动 RPC 服务器，以接受要运行的工作负载。
- 通过指定远程机器的 architecture target（称为交叉编译（cross-compilation）），可以在本地编译程序，然后通过 RPC 在远程机器上运行。

## [讨论](https://discuss.tvm.ai/t/getting-started-running-on-a-remote-machine/4709)
